In [ ]:
import cv2
import os
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from PIL import Image

video_path = r"D:\Research\Git\Image-Processing\Video\testVideo2.mp4"
output_folder = r'D:\Extracted frames'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

cap = cv2.VideoCapture(video_path)
frame_count = 0

model = hub.load("https://tfhub.dev/captain-pool/esrgan-tf2/1")

def enhance_image(image):
    lr_image = tf.convert_to_tensor(image, dtype=tf.float32)
    lr_image = tf.expand_dims(lr_image, 0)
    sr_image = model(lr_image)
    sr_image = tf.squeeze(sr_image)
    sr_image = tf.clip_by_value(sr_image, 0, 255)
    sr_image = tf.cast(sr_image, tf.uint8)
    return sr_image.numpy()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    enhanced_frame_rgb = enhance_image(frame_rgb)

    enhanced_frame_bgr = cv2.cvtColor(enhanced_frame_rgb, cv2.COLOR_RGB2BGR)

    frame_filename = os.path.join(output_folder, f"frame_{frame_count:04d}.png")
    cv2.imwrite(frame_filename, enhanced_frame_bgr)

    frame_count += 1

cap.release()
print(f"Extracted and enhanced {frame_count} frames.")
